In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
import selenium
from selenium.webdriver import Firefox, Chrome, Remote
from selenium.webdriver.common.proxy import *
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
from time import sleep

In [114]:
def get_abstract(name, page):
    '''
    Функция возвращает абстакт для журнала name
    '''
    def nature(page):
        return page.findAll('p')[5].text
    def jetp(page):
        return page.findAll('p')[3].text
    func = {
        'NATURE_PHYSICS' : nature,
        'JETP' : jetp,
        'JETP_LETTERS' : jetp
    }
    return func[name](page)

def get_articles_adress(page):
    '''
    Возвращает список адресов всех статей
    '''
    articles_adress = []
    id_str = '/item.asp?id='
    for i in page.findAll('a'):
        try:
            if id_str in i['href']:
                num_id = i['href']
                articles_adress.append('https://elibrary.ru' + num_id)
        except:
            continue
    return articles_adress

def find_data(browser, name, adress):
    '''
    Возвращает list из [названия статьи, авторы, число цитирований, абстракт]
    '''
    soup = BeautifulSoup(browser.page_source, "html.parser")
    sleep(np.random.rand() * 2)
    data = []
    # Добавление Автора и Названия статьи
    for i in soup.findAll('input'):
        try:    
            if not i['value'][0].isdigit() and (i['value'][0] == i['value'][0].upper()):
                data.append(i['value'])
        except:
            continue
    
    if len(data) < 2:
        data.insert(0, 'Incognito')
    # Добавление количества цитирований
    try:
        quot = soup.findAll('a', attrs= {'href':['cit_items.asp?id=' + adress.split('=')[-1]]})[0].text
    except:
        quot = '0'
    data.append(quot)
    # Добавление абстракта
    abstract = soup.find('p').text
    key = 'https://doi'
    if abstract:
        data.append(abstract)
    else:
        for i in soup.findAll('a'):
            try:
                if key in i['href']:
                    adress_r = i['href']
                    break
            except:
                continue
        browser.get(adress_r)
        ppp = BeautifulSoup(browser.page_source, "html.parser")
        abstract_journal = get_abstract(name, ppp)
        sleep(4 * np.random.random())
        browser.back()
        data.append(abstract_journal) #если нет абстракта берет его с сайта журнала
    return data

def run_to_actual_page(page_num,browser,xp_main):
    for i in range(page_num):
        sleep(7)
        search_form = browser.find_element_by_xpath(xp_main)
        search_form.click()                                                                  # Жмякаем на кнопку

def data_frame(name):
    try:
        return pd.read_csv(name,sep="@")
    except:
        return pd.DataFrame(columns=['Authors', 'Title', 'Qoutes', 'Abstract', 'Link'])
    
def parse_journal(name, browser, page_num):
    print(name)
    print()
    columns = ['Authors', 'Title', 'Qoutes', 'Abstract', 'Link']
    articles = data_frame(name)
    xp_main = '//*[@id="pages"]/table/tbody/tr/td[13]/a'
    run_to_actual_page(page_num, browser, xp_main)
    pp = BeautifulSoup(browser.page_source, "html.parser")
    runs = int(pp.find('font',attrs={'color':['#ff0000']}).text) // 20
    print('runs:', runs)
    sleep(5)
    for loop in range(page_num, runs):
        
        search_form = browser.find_element_by_xpath(xp_main)
        page = BeautifulSoup(browser.page_source, "html.parser")                             
        new_id = get_articles_adress(page)                                                   
        if loop % 2:
            sleep(5 + 15 * np.random.random())
        for i in np.random.choice(range(len(new_id)), 15, replace=False):
            if i % 3 == 0:
                sleep(3 * np.random.random() + 1.5 * np.random.random())
            browser.get(new_id[i])                         
            d = find_data(browser, name, new_id[i])
            d.append(new_id[i])                                                                    
            print("___{0}___{1}___".format(loop, i), end='\r')
            sleep(2 + 5 * np.random.random())
            browser.back()
            articles = articles.append(pd.Series(d, index=columns), ignore_index=True)
            articles.to_csv(name, sep="@")
        search_form = browser.find_element_by_xpath(xp_main)
        search_form.click()                                                                  
        sleep(5)
        

In [ ]:
browser = Chrome(executable_path="./chromedriver")   
browser.get(k)
journals = {
    'NATURE_PHYSICS'    : 'https://elibrary.ru/title_items.asp?id=25368',
    'JETP' : 'https://elibrary.ru/title_items.asp?id=7467',
    'JETP_LETTERS' : 'https://elibrary.ru/title_items.asp?id=7468',
    'PHYSICAL_REVIEW_B' : "https://elibrary.ru/title_items.asp?id=21814",
    'PHYSICAL_REVIEW_C' : "https://elibrary.ru/title_items.asp?id=21815",
    'PHYSICAL_REVIEW_LETTERS' : "https://elibrary.ru/title_items.asp?id=21820",
}
for i, k in journals.items():
    browser.get(k)
    r = input()
    parse_journal(i, browser, 0)
    

5
NATURE_PHYSICS

runs: 28


In [113]:
browser = Chrome(executable_path="./chromedriver")   
browser.get('https://elibrary.ru/item.asp?id=26841171')
p = BeautifulSoup(browser.page_source, 'html.parser')
# get_abstract('JETP_LETTERS', p)

In [ ]:
try:
    quot = p.findAll('a', attrs= {'href':['cit_items.asp?id=' + adress.split('=')[-1]]})[0].text
except:
    quot = '0'